In [33]:
!pip install pyTigerGraph
import pyTigerGraph as tg

In [34]:
conn = tg.TigerGraphConnection(host="https://cc-visuals.i.tgcloud.io/", password="pinchoo", graphname="MyGraph")
conn.apiToken = conn.getToken(conn.createSecret()) 

In [35]:
print(conn.gsql('''
USE GRAPH MyGraph
DROP QUERY grabInfectionLocationDetails
CREATE QUERY grabInfectionLocationDetails() FOR GRAPH MyGraph SYNTAX v2 {
TYPEDEF TUPLE <FLOAT lat, FLOAT lon, STRING infcase, STRING province, UINT num_confirmed_cases, UINT population, FLOAT area> INFO;
HeapAccum<INFO> (10000, num_confirmed_cases DESC, population DESC) @@information;
Seed = {City.*};
Res = SELECT tgt FROM Seed:c - (CASE_IN_CITY:e)- InfectionCase:i -(CASE_IN_PROVINCE:e2)-Province:tgt
      ACCUM @@information+=INFO(e.latitude, e.longitude, i.infection_case, tgt.province, i.confirmed, tgt.population, tgt.area);
PRINT @@information;
}
INSTALL QUERY grabInfectionLocationDetails
'''))

Using graph 'MyGraph'
Successfully dropped queries on the graph 'MyGraph': [grabInfectionLocationDetails].
Successfully created queries: [grabInfectionLocationDetails].
Start installing queries, about 1 minute ...
grabInfectionLocationDetails query: curl -X GET 'https://127.0.0.1:9000/query/MyGraph/grabInfectionLocationDetails'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
Select 'm1' as compile server, now connecting ...
Node 'm1' is prepared as compile server.

Query installation finished.


In [36]:
!pip install -q jupyter-dash
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [37]:
res = conn.runInstalledQuery("grabInfectionLocationDetails")[0]["@@information"]
df = pd.DataFrame(res)


In [38]:
fig = px.scatter_mapbox(df, lat="lat", lon="lon", size="num_confirmed_cases", hover_name="infcase", color="population", hover_data=["area"])

In [39]:
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

In [40]:
fig.show()


In [41]:
fig = px.scatter_mapbox(df, lat="lat", lon="lon", size="num_confirmed_cases", hover_name="infcase", color="population", hover_data=["area"])
fig.update_layout(
   mapbox_style="white-bg",
   mapbox_layers=[
      {
         "below": 'traces',
         "sourcetype": "raster",
         "sourceattribution": "United States Geological Survey",
         "source": ["https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"]
      }
   ]
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [42]:
bar = px.bar(df, x="infcase", y="num_confirmed_cases", color="population", barmode='overlay')
bar.show()

In [43]:
bar = px.scatter_3d(df, "num_confirmed_cases", "population", "area")
bar.show()

In [44]:
app = JupyterDash(__name__)

In [45]:
app.layout = html.Div([
   html.H1("COVID-19 Dashboard Demo"),
   dcc.Dropdown(
      id = "map_type",
      options=[
         {"label": "Street Map", "value": "light"},
         {"label": "Geometric Map", "value": "dark"}
      ],
      value="light",
      clearable=False
   ),
   dcc.Graph(id="map_graph"),
   html.Br(),
   dcc.Dropdown(
      id = "graph_options",
      options=[
         {"label": "County vs. Number of Confirmed Cases", "value": "county_cases"},
         {"label": "Number of Confirmed Cases vs. Population vs. Area", "value": "3d_graph"}
      ],
      value="county_cases",
      clearable=False
   ),
   dcc.Graph(id="bar_graph"),
])

In [46]:
@app.callback(
   Output('map_graph', 'figure'),
   Input('map_type', 'value'),)

def update_graph(typ):
  if typ == "light":
      fig = px.scatter_mapbox(df, lat="lat", lon="lon", size="num_confirmed_cases", hover_name="infcase", color="population", hover_data=["area"], custom_data=["num_confirmed_cases", "population", "area"])
      fig.update_layout(mapbox_style="open-street-map")
      fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
  else:
      fig = px.scatter_mapbox(df, lat="lat", lon="lon", size="num_confirmed_cases", hover_name="infcase", color="population", hover_data=["area"], custom_data=["num_confirmed_cases", "population", "area"])
      fig.update_layout(
         mapbox_style="white-bg",
         mapbox_layers=[{
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": ["https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"]
      }])
      fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
  return fig


In [47]:
@app.callback(
   Output('bar_graph', 'figure'),
   Input('map_graph', 'selectedData'),
   Input('graph_options', 'value'),
)
def update_graph(data, graph_type):
   if data:
      if graph_type == "county_cases":
         df = pd.DataFrame({"num_confirmed_cases": [i["customdata"][0] for i in data["points"]], "place": [i["hovertext"] for i in data["points"]], "population": [i["customdata"][1] for i in data["points"]]})
         bar = px.bar(df, x="place", y="num_confirmed_cases", color="population", barmode='overlay')
         return bar
      elif graph_type == "3d_graph":
         df = pd.DataFrame({"num_confirmed_cases": [i["customdata"][0] for i in data["points"]], "population": [i["customdata"][1] for i in data["points"]], "area": [i["customdata"][2] for i in data["points"]]})
         bar = px.scatter_3d(df, "num_confirmed_cases", "population", "area")
         return bar
   else:
      res = conn.runInstalledQuery("grabInfectionLocationDetails")[0]["@@information"]
      df = pd.DataFrame(res)
      if graph_type == "county_cases":
         bar = px.bar(df, x="infcase", y="num_confirmed_cases", color="population", barmode='overlay')
         return bar
      elif graph_type == "3d_graph":
         bar = px.scatter_3d(df, "num_confirmed_cases", "population", "area")
         return bar

In [48]:
app.run_server(mode='external')

Dash app running on:


<IPython.core.display.Javascript object>